In [3]:
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D

In [4]:
# Load APPA-REAL dataset
# from google.colab import drive
# drive.mount("/content/drive", force_remount=True)
data_dir = "/Users/home/Downloads/appa-real-release/"

labels_path = os.path.join(data_dir, "gt_avg_train.csv")  # Adjust based on dataset

# Read CSV containing file names and age labels
df = pd.read_csv(labels_path)

# Load and preprocess images
images = []
ages = []
for index, row in df.iterrows():
    image_path = os.path.join(data_dir, "train", row['file_name'])  # Adjust folder structure
    if os.path.exists(image_path):
        image = cv2.imread(image_path)
        image = cv2.resize(image, (224, 224))  # Resize for CNN
        images.append(image)
        ages.append(row['apparent_age_avg'])  # Use the average apparent age

images = np.array(images) / 255.0  # Normalize pixel values
ages = np.array(ages)

# Split into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(images, ages, test_size=0.2, random_state=42)

In [5]:
# Load pre-trained MobileNetV2
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights="imagenet")

# Freeze the base model
base_model.trainable = False

# Build the full model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(1)  # Regression output for predicting age
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the model
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=10,
    batch_size=32
)

# Save the model
model.save("/Users/home/Downloads/AgeModels/age_detected_model.keras")

Epoch 1/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 29s 308ms/step - loss: 452.9085 - mae: 16.4513 - val_loss: 165.0536 - val_mae: 10.1100
Epoch 2/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 28s 336ms/step - loss: 183.9905 - mae: 10.3663 - val_loss: 146.6058 - val_mae: 9.4922
Epoch 3/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 24s 288ms/step - loss: 156.9636 - mae: 9.7970 - val_loss: 139.5662 - val_mae: 9.0879
Epoch 4/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 26s 319ms/step - loss: 156.2671 - mae: 9.5437 - val_loss: 142.8239 - val_mae: 9.1061
Epoch 5/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 25s 303ms/step - loss: 143.2968 - mae: 9.2012 - val_loss: 130.5045 - val_mae: 8.7766
Epoch 6/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 27s 331ms/step - loss: 134.2624 - mae: 8.9745 - val_loss: 129.6397 - val_mae: 8.7018
Epoch 7/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 26s 311ms/step - loss: 131.4221 - mae: 8.9241 - val_loss: 128.8203 - val_mae: 8.7142
Epoch 8/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 26s 315ms/step - loss: 135.9735 - mae: 8.9957 - val_loss: 125.9212 - val_mae: 8.5515
Epoch 9/10
83

In [ ]:
# Load the trained model
model = tf.keras.models.load_model("/Users/home/Downloads/AgeModels/age_detected_model.keras")

# Load OpenCV's Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def predict_age(image_path):
    # Load and preprocess the image
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    for (x, y, w, h) in faces:
        # Extract the face ROI
        face = image[y:y+h, x:x+w]
        face = cv2.resize(face, (224, 224)) / 255.0
        face = np.expand_dims(face, axis=0)

        # Predict age
        age = model.predict(face)[0][0]
        age_text = f"Age: {int(age)}"

        # Draw a rectangle and label on the image
        cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.putText(image, age_text, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    # Display the result
    cv2.imshow("Age Detection", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Test the function
predict_age("/Users/home/Downloads/appa-real-release/test/005613.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 936ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
